# 🧠 InsightSpike-AI Google Colab Demo

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

This notebook demonstrates InsightSpike-AI in Google Colab environment.

⚡ **GPU Runtime Recommended**: Runtime > Change runtime type > GPU

## 🕒 Setup Time Estimates
- **Fast Setup**: ~3-5 minutes (with PyTorch Geometric timeout protection)
- **Standard Setup**: ~8-12 minutes (full installation)
- **Debug Setup**: ~15-20 minutes (with detailed logging)

## 🚀 Setup Options
Choose from 3 setup options:

## 🔧 Environment Setup

In [ ]:
# Repository cloning
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI

In [ ]:
# 🧠 InsightSpike-AI Colab Setup (Simple CLI Version)
# Run each cell step-by-step for clear progress tracking

print("🚀 InsightSpike-AI Colab Setup")
print("=" * 50)
print("\n📋 Step-by-step setup:")
print("1. Run repository clone cell")
print("2. Run basic dependencies cell")
print("3. Choose ONE setup cell below:")
print("   • Fast setup (3-5 min, recommended)")
print("   • Minimal setup (1 min, basic features)")
print("   • Debug setup (detailed logging)")
print("4. Run validation cell")
print("\n⚡ Tip: Each cell shows CLI output for easy debugging")

In [ ]:
# Environment setup (GPU support)
# Grant execution permissions to necessary script files
!chmod +x scripts/colab/setup_colab.sh
!chmod +x scripts/setup/setup.sh
!chmod +x scripts/setup/refactor_prepare.sh

# Grant execution permissions to Python scripts as needed
!find scripts/ -name "*.py" -exec chmod +x {} \;

print("📋 Setting up file permissions...")

!chmod +x scripts/colab/*.sh
!chmod +x scripts/setup/*.sh
!find scripts/ -name "*.py" -exec chmod +x {} \;

print("✅ Permissions set successfully")
print("\n📍 Ready for setup - choose one of the setup cells below")

In [ ]:
# 🚀 Setup Options (Choose ONE)

# Run **only one** of the following setup cells based on your needs:

# ================================================
# EDIT HERE: Enter 1, 2, 3, or 4
SETUP_CHOICE = 1  # ← Change this number!
# ================================================

import os
import subprocess
import time

# Setup options definition
setup_commands = {
    1: {
        'name': '⚡ Fast Setup',
        'script': 'scripts/colab/setup_colab_fast.sh',
        'description': 'Recommended: Development & Testing',
        'time': '3-5 minutes'
    },
    2: {
        'name': '🚀 Minimal Setup', 
        'script': 'scripts/colab/setup_colab_minimal.sh',
        'description': 'Ultra-fast: Basic features only',
        'time': 'Under 60 seconds'
    },
    3: {
        'name': '🔍 Debug Setup',
        'script': 'scripts/colab/setup_colab_debug.sh', 
        'description': 'Troubleshooting with detailed logs',
        'time': '15-20 minutes'
    },
    4: {
        'name': '🔧 Full Setup',
        'script': 'scripts/colab/setup_colab.sh',
        'description': 'Complete legacy setup',
        'time': '8-15 minutes'
    }
}

def print_progress(step, total_steps, description):
    """Display text-based progress"""
    percentage = (step / total_steps) * 100
    progress_bar = "█" * int(percentage // 5) + "░" * (20 - int(percentage // 5))
    print(f"\n[{progress_bar}] {percentage:.1f}% - {description}")

if SETUP_CHOICE in setup_commands:
    selected = setup_commands[SETUP_CHOICE]
    
    print(f"\n🎯 Selected: {selected['name']}")
    print(f"Description: {selected['description']}")
    print(f"Expected time: {selected['time']}")
    print("=" * 60)
    
    # Pre-setup preparation
    print_progress(1, 5, "Setting execution permissions...")
    time.sleep(0.5)
    
    # Set execution permissions
    os.system(f"chmod +x {selected['script']}")
    
    print_progress(2, 5, "Launching setup script...")
    print(f"\n🚀 Executing: {selected['script']}")
    print(f"📅 Start time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️ Expected completion: {selected['time']}")
    print("\n📋 Setup log:")
    print("-" * 40)
    
    # Execute setup
    start_time = time.time()
    result = os.system(f"./{selected['script']}")
    
    # Manual FAISS installation if the setup script missed it
    if result == 0:  # If setup was successful, ensure FAISS is installed
        print("\n🔧 Ensuring FAISS installation...")
        print("📋 Installing CUDA runtime libraries...")
        
        # Install CUDA runtime dependencies first
        cuda_result = os.system("pip install -q nvidia-cuda-runtime-cu12 nvidia-cublas-cu12")
        if cuda_result == 0:
            print("✅ CUDA runtime libraries installed successfully")
            
            # Try installing FAISS-GPU-CU12 with explicit version
            print("📋 Installing FAISS-GPU-CU12...")
            faiss_result = os.system("pip install 'faiss-gpu-cu12>=1.11.0' --timeout 180")
            if faiss_result == 0:
                print("✅ FAISS-GPU-CU12 installed successfully")
            else:
                print("⚠️ FAISS-GPU-CU12 failed, installing FAISS-CPU as fallback...")
                cpu_result = os.system("pip install faiss-cpu")
                if cpu_result == 0:
                    print("✅ FAISS-CPU installed successfully")
                else:
                    print("❌ Both FAISS-GPU and FAISS-CPU installation failed")
        else:
            print("⚠️ CUDA runtime installation failed, installing FAISS-CPU...")
            cpu_result = os.system("pip install faiss-cpu")
            if cpu_result == 0:
                print("✅ FAISS-CPU installed successfully")
            else:
                print("❌ FAISS-CPU installation failed")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print_progress(5, 5, "Setup validation complete")
    
    if result == 0:
        print(f"\n✅ {selected['name']} Complete!")
        print(f"Execution time: {minutes}m {seconds}s")
        print("🎉 Setup completed successfully")
    else:
        print(f"\n❌ Setup Error")
        print(f"Execution time: {minutes}m {seconds}s")
        print("🔍 Recommend Debug Setup (3) for detailed investigation")
    
    print(f"\n📊 Setup completed - Execution time: {minutes}m {seconds}s")
    print("\n🔍 Next step: Run the validation cell below to check installation status")
    
else:
    print(f"\n❌ Invalid selection: {SETUP_CHOICE}")
    print("Valid choices: 1 (Fast), 2 (Minimal), 3 (Debug), 4 (Full)")
    print("👆 Change SETUP_CHOICE to 1, 2, 3, or 4 and re-run")

In [ ]:
# ⚡ OPTION 1: Fast Setup (Recommended)
# Time: 3-5 minutes, with timeout protection

print("⚡ Starting Fast Setup...")
print("Time estimate: 3-5 minutes")
print("Features: Timeout protection, full functionality")
print("="*50)

# Core dependencies
print("📋 Step 1/4: Installing core dependencies...")
!pip install -q torch torchvision numpy pandas matplotlib seaborn plotly
print("✅ Core dependencies installed")

# FAISS with timeout protection
print("\n📋 Step 2/4: Installing FAISS (with timeout protection)...")
import subprocess
import signal

def timeout_handler(signum, frame):
    raise TimeoutError("FAISS installation timeout")

try:
    # Set 3-minute timeout for FAISS
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(180)  # 3 minutes
    
    result = subprocess.run(["pip", "install", "-q", "faiss-gpu-cu12"], 
                           capture_output=True, text=True, timeout=180)
    signal.alarm(0)  # Cancel alarm
    
    if result.returncode == 0:
        print("✅ FAISS-GPU installed successfully")
    else:
        print("⚠️ FAISS-GPU failed, installing CPU version...")
        !pip install -q faiss-cpu
        print("✅ FAISS-CPU installed as fallback")
except (TimeoutError, subprocess.TimeoutExpired):
    print("⚠️ FAISS installation timeout, installing CPU version...")
    !pip install -q faiss-cpu
    print("✅ FAISS-CPU installed as fallback")

# PyTorch Geometric with protection
print("\n📋 Step 3/4: Installing PyTorch Geometric...")
try:
    !pip install -q torch-geometric --timeout 120
    print("✅ PyTorch Geometric installed")
except:
    print("⚠️ PyTorch Geometric installation skipped (optional)")

# Additional dependencies
print("\n📋 Step 4/4: Installing additional dependencies...")
!pip install -q networkx scikit-learn transformers datasets
print("✅ Additional dependencies installed")

print("\n🎉 Fast Setup Complete!")
print("Next: Run the validation cell below")

In [ ]:
# 🚀 OPTION 2: Minimal Setup (Ultra-fast)
# Time: Under 1 minute, basic features only

print("🚀 Starting Minimal Setup...")
print("Time estimate: Under 1 minute")
print("Features: Basic functionality only")
print("="*50)

# Essential dependencies only
print("📋 Installing essential dependencies...")
!pip install -q torch numpy pandas matplotlib
print("✅ Essential dependencies installed")

# FAISS CPU (fast and reliable)
print("\n📋 Installing FAISS-CPU...")
!pip install -q faiss-cpu
print("✅ FAISS-CPU installed")

# Basic ML libraries
print("\n📋 Installing basic ML libraries...")
!pip install -q scikit-learn transformers
print("✅ Basic ML libraries installed")

print("\n🎉 Minimal Setup Complete!")
print("⚠️ Note: PyTorch Geometric not installed (optional feature)")
print("Next: Run the validation cell below")

In [ ]:
# 🔍 OPTION 3: Debug Setup (Detailed logging)
# Time: 15-20 minutes, with comprehensive logging

print("🔍 Starting Debug Setup...")
print("Time estimate: 15-20 minutes")
print("Features: Detailed logging for troubleshooting")
print("="*50)

import subprocess
import sys

def run_with_log(cmd, description):
    """Run command with detailed logging"""
    print(f"\n📋 {description}")
    print(f"Command: {cmd}")
    
    try:
        result = subprocess.run(cmd.split(), capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print(f"✅ Success: {description}")
            if result.stdout:
                print(f"Output: {result.stdout[:200]}..." if len(result.stdout) > 200 else f"Output: {result.stdout}")
        else:
            print(f"❌ Failed: {description}")
            print(f"Error: {result.stderr}")
        
        return result.returncode == 0
    except subprocess.TimeoutExpired:
        print(f"⏰ Timeout: {description}")
        return False
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# Detailed installation with logging
print("📋 Step 1/5: Core dependencies...")
run_with_log("pip install torch torchvision numpy pandas", "Installing PyTorch and basic libraries")

print("\n📋 Step 2/5: Visualization libraries...")
run_with_log("pip install matplotlib seaborn plotly", "Installing visualization libraries")

print("\n📋 Step 3/5: FAISS installation...")
if not run_with_log("pip install faiss-gpu-cu12", "Installing FAISS-GPU"):
    print("⚠️ Falling back to FAISS-CPU...")
    run_with_log("pip install faiss-cpu", "Installing FAISS-CPU")

print("\n📋 Step 4/5: PyTorch Geometric...")
if not run_with_log("pip install torch-geometric", "Installing PyTorch Geometric"):
    print("⚠️ PyTorch Geometric installation failed (optional)")

print("\n📋 Step 5/5: Additional ML libraries...")
run_with_log("pip install scikit-learn transformers datasets networkx", "Installing ML libraries")

print("\n🎉 Debug Setup Complete!")
print("Check the logs above for any issues")
print("Next: Run the validation cell below")

In [ ]:
# ## 📋 Setup Tips
# 
# **If setup fails:**
# - Try Minimal Setup first (fastest, most reliable)
# - Use Debug Setup for detailed error logs
# - Check GPU runtime is enabled in Colab
# 
# **Next step:** Run validation cell below

In [ ]:
# 📊 Quick System Check
# Optional: Run this to test basic system functionality

print("📊 Running basic system check...")

# Quick imports test
try:
    import numpy as np
    import pandas as pd
    print("✅ NumPy and Pandas: OK")
except Exception as e:
    print(f"❌ NumPy/Pandas: {e}")

# Quick computation test
try:
    x = np.random.random((1000, 100))
    result = np.mean(x, axis=0)
    print(f"✅ Computation test: OK ({len(result)} features)")
except Exception as e:
    print(f"❌ Computation test: {e}")

print("🎉 Basic check complete!")
print("Next: Run the setup validation cell below")

## 🔍 Setup Validation and Troubleshooting

In [ ]:
# 🔍 Setup Validation (CLI Style)
# Run this after completing one of the setup options above

print("🔍 Validating Setup...")
print("=" * 40)

# Test 1: PyTorch
print("\n📋 Test 1/5: PyTorch")
try:
    import torch
    cuda_ok = torch.cuda.is_available()
    print(f"  ✅ PyTorch: {torch.__version__}")
    print(f"  GPU: {'YES' if cuda_ok else 'NO (CPU only)'}")
except Exception as e:
    print(f"  ❌ PyTorch: Failed - {e}")

# Test 2: FAISS
print("\n📋 Test 2/5: FAISS")
try:
    import faiss
    gpu_count = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
    print(f"  ✅ FAISS: {faiss.__version__}")
    print(f"  GPU: {gpu_count} devices available")
except Exception as e:
    print(f"  ❌ FAISS: Failed - {e}")

# Test 3: PyTorch Geometric (optional)
print("\n📋 Test 3/5: PyTorch Geometric")
try:
    import torch_geometric
    print(f"  ✅ PyG: {torch_geometric.__version__}")
except Exception as e:
    print(f"  ⚠️ PyG: Not available (OK for basic setup)")

# Test 4: HuggingFace
print("\n📋 Test 4/5: HuggingFace")
try:
    import transformers
    print(f"  ✅ Transformers: {transformers.__version__}")
except Exception as e:
    print(f"  ❌ Transformers: Failed - {e}")

# Test 5: InsightSpike Core
print("\n📋 Test 5/5: InsightSpike Core")
try:
    import sys
    sys.path.insert(0, 'src')
    from insightspike.core.config import get_config
    print(f"  ✅ Core: Ready")
except Exception as e:
    print(f"  ⚠️ Core: {e}")

print("\n🎉 Validation Complete!")
print("If tests failed, try the Debug Setup option")

In [ ]:
# 🚀 FAISS Performance Test (Simple CLI Version)
# Test FAISS performance with simple output

print("🚀 FAISS Performance Test")
print("=" * 30)

try:
    import faiss
    import numpy as np
    import time
    
    # Simple test data
    dimension = 128
    n_vectors = 1000
    n_queries = 10
    
    print(f"Test data: {n_vectors} vectors, {dimension}D")
    
    # Generate test data
    vectors = np.random.random((n_vectors, dimension)).astype('float32')
    queries = np.random.random((n_queries, dimension)).astype('float32')
    
    # CPU test
    print("\n📋 CPU test...")
    cpu_index = faiss.IndexFlatL2(dimension)
    
    start = time.time()
    cpu_index.add(vectors)
    distances, indices = cpu_index.search(queries, 5)
    cpu_time = time.time() - start
    
    print(f"  CPU time: {cpu_time:.3f}s")
    print(f"  Results: Found {len(indices[0])} neighbors")
    
    # GPU test (if available)
    gpu_count = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
    
    if gpu_count > 0:
        print("\n📋 GPU test...")
        try:
            res = faiss.StandardGpuResources()
            gpu_index = faiss.index_cpu_to_gpu(res, 0, faiss.IndexFlatL2(dimension))
            
            start = time.time()
            gpu_index.add(vectors)
            distances, indices = gpu_index.search(queries, 5)
            gpu_time = time.time() - start
            
            speedup = cpu_time / gpu_time if gpu_time > 0 else 0
            print(f"  GPU time: {gpu_time:.3f}s")
            print(f"  Speedup: {speedup:.1f}x")
            
            if speedup > 1.5:
                print("  ✅ GPU acceleration working well!")
            else:
                print("  ⚠️ GPU acceleration minimal (normal for small data)")
                
        except Exception as e:
            print(f"  ❌ GPU test failed: {e}")
    else:
        print("\n⚠️ GPU not available - using CPU only")
    
    print("\n🎉 FAISS test complete!")
    
except Exception as e:
    print(f"❌ FAISS test failed: {e}")
    print("Try running one of the setup cells above first")

## 📊 Data Preparation

In [ ]:
# CLIコマンドアクセス確認
print("\n💻 CLI Command Access Validation:")

# Poetry CLIアクセステスト
try:
    import subprocess
    result = subprocess.run(['poetry', 'run', 'python', '-c', 'import insightspike.cli; print("CLI module accessible")'], 
                          capture_output=True, text=True, cwd='.')
    if result.returncode == 0:
        print("✅ Poetry CLI access: Available")
        print(f"   Output: {result.stdout.strip()}")
    else:
        print(f"❌ Poetry CLI access failed: {result.stderr}")
except Exception as e:
    print(f"❌ Poetry CLI test error: {e}")

# 直接アクセステスト
try:
    import sys
    import os
    old_path = sys.path.copy()
    if 'src' not in sys.path:
        sys.path.insert(0, 'src')
    
    import insightspike.cli
    print("✅ Direct CLI module access: Available")
    
    # PYTHONPATHテスト
    os.environ['PYTHONPATH'] = 'src'
    result = subprocess.run(['python', '-c', 'import insightspike.cli; print("Direct access works")'], 
                          capture_output=True, text=True, env=os.environ)
    if result.returncode == 0:
        print("✅ PYTHONPATH direct access: Available")
    else:
        print(f"⚠️ PYTHONPATH access issue: {result.stderr}")
    
    sys.path = old_path
except Exception as e:
    print(f"❌ Direct CLI access error: {e}")

print("\n🚀 Recommended CLI usage:")
print("   🎆 Primary:   !PYTHONPATH=src poetry run python -m insightspike.cli [command]")
print("   🔄 Fallback:  !PYTHONPATH=src python -m insightspike.cli [command]")

In [ ]:
# データの準備（サンプルデータ作成）
# 必要なディレクトリを作成
!mkdir -p data/raw data/processed data/embedding data/models

# サンプルデータファイルを作成
with open('data/raw/test_sentences.txt', 'w', encoding='utf-8') as f:
    f.write("""The aurora borealis is caused by charged particles from the sun interacting with Earth's magnetic field.
Quantum entanglement is a phenomenon where particles become correlated in ways that defy classical physics.
Artificial intelligence uses machine learning algorithms to process data and make predictions.
The human brain contains billions of neurons that communicate through synapses.
Photosynthesis converts sunlight into chemical energy in plants.
DNA contains the genetic instructions for all living organisms.
Gravity is a fundamental force that attracts objects with mass toward each other.
Evolution explains how species change over time through natural selection.
""")

print("✅ Data directories created")
print("✅ Sample data created in data/raw/test_sentences.txt")

In [ ]:
# Build episodic memory
# Method 1: Poetry run (recommended)
!PYTHONPATH=src poetry run python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 2: Direct execution (alternative when Poetry has issues)
# !PYTHONPATH=src python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 3: Using Makefile
# !make embed

In [ ]:
# Build similarity graph
# Method 1: Poetry run (recommended)
!PYTHONPATH=src poetry run python -m insightspike.cli graph

# Method 2: Direct execution (alternative when Poetry has issues)
# !PYTHONPATH=src python -m insightspike.cli graph

## 🚀 Demo Execution

In [ ]:
# Simple test execution
# Method 1: Poetry run (recommended)
!PYTHONPATH=src poetry run python -m insightspike.cli loop "What is quantum entanglement?"

# Method 2: Direct execution (alternative when Poetry has issues)
# !PYTHONPATH=src python -m insightspike.cli loop "What is quantum entanglement?"

# 🚀 Interactive Demo Execution
# Select a question to verify InsightSpike-AI operation

import time
from IPython.display import display, HTML, clear_output

# Demo question presets
demo_questions = {
    1: {
        'question': 'What is quantum entanglement?',
        'category': '🔬 Physics',
        'description': 'About quantum entanglement - complex physics concept'
    },
    2: {
        'question': 'How does artificial intelligence work?',
        'category': '🤖 AI/Technology', 
        'description': 'How AI works - technology explanation'
    },
    3: {
        'question': 'What causes the aurora borealis?',
        'category': '🌌 Natural Phenomena',
        'description': 'Aurora causes - natural phenomenon mechanism'
    },
    4: {
        'question': 'How does photosynthesis work?',
        'category': '🌱 Biology',
        'description': 'Photosynthesis process - basic biology concept'
    },
    5: {
        'question': 'Custom Question',
        'category': '✏️ Custom',
        'description': 'Enter your own question'
    }
}

def print_question_menu():
    """Print question selection menu in text format"""
    print("🧠 InsightSpike-AI Demo Question Selection")
    print("=" * 60)
    print()
    
    for q_id, q_data in demo_questions.items():
        category_icon = "✏️" if q_id == 5 else "📋"
        border_char = "~" if q_id == 5 else "-"
        
        print(f"{category_icon} {q_id}. {q_data['question']}")
        print(f"   Category: {q_data['category']}")
        print(f"   Description: {q_data['description']}")
        print(border_char * 50)
        print()
    
    print("📋 How to Use:")
    print("   1. Choose a question number (1-5)")
    print("   2. Edit DEMO_CHOICE in the cell below")
    print("   3. If choosing 5 (Custom), set CUSTOM_QUESTION")
    print("   4. Run the execution cell")
    print()

# Display question selection menu
print_question_menu()

print("\n👇 Select DEMO_CHOICE in the cell below to run the demo")

In [ ]:
# 🎯 Simple Demo Execution
# For users who want the quickest test

# Just run this cell to test with a pre-selected question
# No need to edit any variables

print("🎯 Simple Demo: Testing InsightSpike-AI")
print("Question: What is quantum entanglement?")
print("=" * 40)

import subprocess
import time

start_time = time.time()

try:
    print("🔄 Running demo...")
    result = subprocess.run([
        'poetry', 'run', 'python', '-m', 'insightspike.cli', 'loop', 
        "What is quantum entanglement?"
    ], capture_output=True, text=True, env={'PYTHONPATH': 'src'}, timeout=120)
    
    execution_time = time.time() - start_time
    
    if result.returncode == 0:
        print(f"✅ Demo successful! ({execution_time:.1f}s)")
        print("\n📋 Results:")
        print("-" * 30)
        print(result.stdout)
        if result.stderr:
            print(f"\n⚠️ Warnings: {result.stderr}")
    else:
        print(f"❌ Demo failed ({execution_time:.1f}s)")
        print(f"Error: {result.stderr}")
        
except subprocess.TimeoutExpired:
    print(f"⏱️ Demo timed out after 120 seconds")
except Exception as e:
    print(f"❌ Demo error: {e}")

print(f"\nTotal time: {execution_time:.1f}s")

In [ ]:
# ✨ Demo Execution & Results Display
# Execute the question selected from the menu above

# ================================================
# EDIT HERE: Enter 1, 2, 3, 4, or 5
DEMO_CHOICE = 1  # ← Change this number!
CUSTOM_QUESTION = ""  # Only used when DEMO_CHOICE = 5
# ================================================

import subprocess
import time

# Demo question definitions (same as cell above)
demo_questions = {
    1: {
        'question': 'What is quantum entanglement?',
        'category': '🔬 Physics',
        'description': 'About quantum entanglement - complex physics concept'
    },
    2: {
        'question': 'How does artificial intelligence work?',
        'category': '🤖 AI/Technology', 
        'description': 'How AI works - technology explanation'
    },
    3: {
        'question': 'What causes the aurora borealis?',
        'category': '🌌 Natural Phenomena',
        'description': 'Aurora causes - natural phenomenon mechanism'
    },
    4: {
        'question': 'How does photosynthesis work?',
        'category': '🌱 Biology',
        'description': 'Photosynthesis process - basic biology concept'
    }
}

def print_demo_status(status, question, category, execution_time=None, error=None):
    """Print demo execution status in simple text format"""
    if status == "running":
        print(f"\n🔄 Running Demo...")
        print("=" * 50)
        print(f"Question: {question}")
        print(f"Category: {category}")
        print("Status: Processing...")
        print("=" * 50)
    elif status == "success":
        print(f"\n✅ Demo Execution Successful!")
        print("=" * 50)
        print(f"Question: {question}")
        print(f"Category: {category}")
        print(f"Execution time: {execution_time:.1f} seconds")
        print("=" * 50)
    elif status == "error":
        print(f"\n❌ Demo Execution Failed")
        print("=" * 50)
        print(f"Question: {question}")
        print(f"Category: {category}")
        print(f"Error: {error}")
        print("=" * 50)

# Selective execution
if DEMO_CHOICE in demo_questions:
    selected_q = demo_questions[DEMO_CHOICE]
    question = selected_q['question']
    category = selected_q['category'] 
    description = selected_q['description']
elif DEMO_CHOICE == 5 and CUSTOM_QUESTION:
    question = CUSTOM_QUESTION
    category = '✏️ Custom'
    description = 'Custom question'
else:
    error_msg = f"Invalid selection: {DEMO_CHOICE}"
    print(f"\n❌ {error_msg}")
    print("=" * 50)
    print("Valid choices: 1-4 (presets), 5 (custom)")
    print("For custom, please set CUSTOM_QUESTION")
    print("=" * 50)
    question = None

if question:
    # Execution start notification
    print_demo_status("running", question, category)
    
    start_time = time.time()
    
    try:
        # Method 1: Poetry run (recommended)
        print(f"\n📋 Executing with Poetry CLI...")
        result = subprocess.run([
            'poetry', 'run', 'python', '-m', 'insightspike.cli', 'loop', question
        ], capture_output=True, text=True, env={'PYTHONPATH': 'src'}, timeout=120)
        
        if result.returncode == 0:
            execution_time = time.time() - start_time
            print_demo_status("success", question, category, execution_time)
            print(f"\n📋 Results:")
            print("-"*40)
            print(result.stdout)
            if result.stderr:
                print(f"\n⚠️ Warnings:")
                print(result.stderr)
        else:
            # Method 2: Direct execution (fallback)
            print(f"\n📋 Trying direct execution...")
            result2 = subprocess.run([
                'python', '-m', 'insightspike.cli', 'loop', question
            ], capture_output=True, text=True, env={'PYTHONPATH': 'src'}, timeout=120)
            
            if result2.returncode == 0:
                execution_time = time.time() - start_time
                print_demo_status("success", question, category, execution_time)
                print(f"\n📋 Results:")
                print("-"*40)
                print(result2.stdout)
                if result2.stderr:
                    print(f"\n⚠️ Warnings:")
                    print(result2.stderr)
            else:
                execution_time = time.time() - start_time
                error_msg = result2.stderr or result.stderr or "Unknown error"
                print_demo_status("error", question, category, None, error_msg[:100])
                print(f"\n📄 Error Details:")
                print(f"Poetry error: {result.stderr}")
                print(f"Direct error: {result2.stderr}")
    
    except subprocess.TimeoutExpired:
        execution_time = time.time() - start_time
        print_demo_status("error", question, category, None, "Timeout (120 seconds)")
        print(f"⏱️ Execution timed out after 120 seconds")
    except Exception as e:
        execution_time = time.time() - start_time
        print_demo_status("error", question, category, None, str(e))
        print(f"❌ Unexpected error: {e}")
    
    print(f"\n🕰️ Total time: {execution_time:.1f} seconds")

else:
    print("👆 Please set the correct DEMO_CHOICE above and re-run")

## 🔍 Interactive Testing

In [ ]:
# 🔬 Advanced Interactive Testing & Analysis
# Detailed system testing and visualization in Python environment

import sys
import os
import time
import json
import matplotlib.pyplot as plt
import numpy as np

# Environment setup
sys.path.insert(0, 'src')
os.environ['PYTHONPATH'] = 'src'

def print_analysis_result(title, status, details, metrics=None):
    """Print analysis result in simple text format"""
    status_icon = "✅" if status == "Success" else "❌" if status == "Failed" or status == "Error" else "📋"
    
    print(f"\n{status_icon} {title}")
    print("=" * 40)
    print(f"Status: {status}")
    print(f"Details: {details}")
    
    if metrics:
        print("\nMetrics:")
        for key, value in metrics.items():
            print(f"  {key}: {value}")
    print("-" * 40)

print("🔬 InsightSpike-AI Advanced Interactive Testing Started")
print("=" * 50)

# System initialization test
print("\n🔋 Step 1: System Initialization Test")
try:
    from insightspike.core.layers.layer2_memory_manager import L2MemoryManager
    from insightspike.core.agents.main_agent import MainAgent
    
    init_start = time.time()
    agent = MainAgent()
    init_success = agent.initialize()
    init_time = time.time() - init_start
    
    if init_success:
        memory_stats = agent.l2_memory.get_memory_stats()
        total_episodes = memory_stats.get('total_episodes', 0)
        
        print_analysis_result(
            "🧠 System Initialization",
            "Success", 
            f"Agent initialized successfully",
            {
                "Init Time": f"{init_time:.3f}s",
                "Episodes": f"{total_episodes}",
                "Memory State": "Normal",
                "GPU Support": "Enabled" if hasattr(agent.l2_memory, 'gpu_accelerated') else "Unknown"
            }
        )
        
        print(f"✅ Agent initialization successful ({init_time:.3f}s)")
        print(f"✅ Memory loaded successfully: {total_episodes} episodes")
    else:
        print_analysis_result(
            "🧠 System Initialization",
            "Failed", 
            "Agent initialization failed",
            {"Init Time": f"{init_time:.3f}s", "Error": "Init failed"}
        )
        print(f"❌ Agent initialization failed")
        agent = None

except Exception as e:
    print_analysis_result(
        "🧠 System Initialization",
        "Error", 
        f"Setup error: {str(e)[:50]}...",
        {"Error Type": type(e).__name__}
    )
    print(f"❌ Setup error: {e}")
    agent = None

# Question processing test
if agent:
    print("\n🔋 Step 2: Question Processing Test")
    
    # Test question set
    test_questions = [
        "What causes the aurora borealis?",
        "How does artificial intelligence work?",
        "What is quantum entanglement?"
    ]
    
    question_results = []
    
    for i, question in enumerate(test_questions):
        print(f"\n📋 Question {i+1}: {question}")
        
        try:
            process_start = time.time()
            result = agent.process_question(question, verbose=False)
            process_time = time.time() - process_start
            
            # Result analysis
            response = result.get('response', 'No response')
            reasoning_quality = result.get('reasoning_quality', 0)
            spike_detected = result.get('spike_detected', False)
            total_cycles = result.get('total_cycles', 0)
            
            question_results.append({
                'question': question,
                'process_time': process_time,
                'reasoning_quality': reasoning_quality,
                'spike_detected': spike_detected,
                'total_cycles': total_cycles,
                'response_length': len(response)
            })
            
            # Quality assessment
            quality_status = "High Quality" if reasoning_quality > 0.7 else "Medium" if reasoning_quality > 0.5 else "Low Quality"
            
            print_analysis_result(
                f"💬 Question {i+1} Results",
                quality_status,
                f"{question[:50]}...",
                {
                    "Process Time": f"{process_time:.3f}s",
                    "Quality Score": f"{reasoning_quality:.3f}",
                    "Insight Detection": "Yes" if spike_detected else "No",
                    "Cycles": f"{total_cycles}"
                }
            )
            
            print(f"  ✅ Processing successful ({process_time:.3f}s)")
            print(f"  📋 Quality: {reasoning_quality:.3f}, Insight: {spike_detected}, Cycles: {total_cycles}")
            
        except Exception as e:
            print_analysis_result(
                f"💬 Question {i+1} Results",
                "Error",
                f"{question[:50]}...",
                {"Error": str(e)[:30]}
            )
            print(f"  ❌ Processing error: {e}")
    
    # Performance analysis
    if question_results:
        print("\n🔋 Step 3: Performance Analysis")
        
        avg_time = np.mean([r['process_time'] for r in question_results])
        avg_quality = np.mean([r['reasoning_quality'] for r in question_results])
        total_spikes = sum([r['spike_detected'] for r in question_results])
        avg_cycles = np.mean([r['total_cycles'] for r in question_results])
        
        print_analysis_result(
            "📋 Performance Summary",
            "Complete",
            f"{len(question_results)} questions tested successfully",
            {
                "Avg Process Time": f"{avg_time:.3f}s",
                "Avg Quality": f"{avg_quality:.3f}",
                "Insight Detection": f"{total_spikes}/{len(question_results)}",
                "Avg Cycles": f"{avg_cycles:.1f}"
            }
        )

# Testing summary
print(f"\n" + "=" * 50)
print(f"📋 Advanced Interactive Test Results Summary")
print(f"=" * 50)
print(f"🎉 Advanced interactive testing complete!")
print(f"🕰️ Test completion time: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Visualization
if agent and 'question_results' in locals() and question_results:
    print("\n📋 Performance Visualization")
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('InsightSpike-AI Performance Analysis', fontsize=14)
    
    # Processing time
    process_times = [r['process_time'] for r in question_results]
    ax1.bar(range(len(process_times)), process_times, color='skyblue')
    ax1.set_title('Processing Time (s)')
    ax1.set_xlabel('Question')
    ax1.set_ylabel('Time (s)')
    
    # Quality scores
    quality_scores = [r['reasoning_quality'] for r in question_results]
    ax2.bar(range(len(quality_scores)), quality_scores, color='lightgreen')
    ax2.set_title('Reasoning Quality Score')
    ax2.set_xlabel('Question')
    ax2.set_ylabel('Quality Score')
    ax2.set_ylim(0, 1)
    
    # Cycle count
    cycles = [r['total_cycles'] for r in question_results]
    ax3.bar(range(len(cycles)), cycles, color='lightcoral')
    ax3.set_title('Reasoning Cycles')
    ax3.set_xlabel('Question')
    ax3.set_ylabel('Cycle Count')
    
    # Insight detection
    spike_counts = [1 if r['spike_detected'] else 0 for r in question_results]
    ax4.pie([sum(spike_counts), len(spike_counts) - sum(spike_counts)], 
            labels=['Insight Detected', 'No Insight'], 
            autopct='%1.1f%%', colors=['gold', 'lightgray'])
    ax4.set_title('Insight Detection Rate')
    
    plt.tight_layout()
    plt.show()

print(f"\n🎉 Advanced interactive testing complete!")
print(f"🕰️ Test completion time: {time.strftime('%Y-%m-%d %H:%M:%S')}")

## 🔧 Comprehensive Troubleshooting Guide

### 🚑 Emergency Quick Fixes

**📚 Setup-Related Errors**
```python
# 1. Runtime restart
# Runtime > Restart runtime

# 2. Emergency setup (under 60 seconds)
SETUP_CHOICE = 2  # Minimal Setup

# 3. GPU configuration check
# Runtime > Change runtime type > GPU
```

**🔋 Module Import Errors**
```python
# 1. PYTHONPATH configuration
import sys, os
sys.path.insert(0, 'src')
os.environ['PYTHONPATH'] = 'src'

# 2. Direct installation
!pip install -e .

# 3. Individual package installation
!pip install torch transformers faiss-cpu
```

**🚀 CLI Command Errors**
```python
# For Poetry errors
!PYTHONPATH=src python -m insightspike.cli --help

# For module errors
!PYTHONPATH=src python scripts/run_poc_simple.py "test question"
```

---

### 📊 Detailed Error Diagnosis & Solutions

#### 1. **🔥 CUDA/GPU Related Errors**

**Symptoms**: `CUDA not available`, `GPU acceleration disabled`

**Solution Steps**:
1. **GPU Runtime Check**: Runtime > Change runtime type > GPU
2. **Verification Commands**:
   ```python
   import torch
   print(f"CUDA Available: {torch.cuda.is_available()}")
   print(f"GPU Count: {torch.cuda.device_count()}")
   ```
3. **Fallback**: CPU mode still supports basic functionality

#### 2. **📦 PyTorch Geometric Hangup**

**Symptoms**: Installation stops for 120+ seconds

**Solution Steps**:
1. **Timeout Protection**: Use Fast Setup (1)
2. **Skip**: Use Minimal Setup (2) to test without PyG
3. **Debug**: Use Debug Setup (3) for detailed logs

#### 3. **📋 Data File Errors**

**Symptoms**: `FileNotFoundError`, `Data not found`

**Solution Steps**:
1. **Directory Creation**:
   ```python
   !mkdir -p data/raw data/processed data/embedding
   ```
2. **Recreate Sample Data**: Re-run data preparation cell
3. **Path Verification**:
   ```python
   import os
   print(os.listdir('data/raw'))
   ```

#### 4. **🧠 Memory Errors**

**Symptoms**: `Out of memory`, `Kernel crash`

**Solution Steps**:
1. **Runtime Restart**: Runtime > Restart runtime
2. **Memory Cleanup**:
   ```python
   import gc
   gc.collect()
   ```
3. **Reduce Batch Size**: Test with smaller datasets

#### 5. **🔗 Network Timeouts**

**Symptoms**: Package download failures

**Solution Steps**:
1. **Extend Timeout**:
   ```python
   !pip install --timeout 300 [package_name]
   ```
2. **Use Mirror**:
   ```python
   !pip install -i https://pypi.org/simple/ [package_name]
   ```
3. **Staged Installation**: Prioritize core packages

---

### 📊 Performance Optimization

#### 🚀 **Speed Optimization Tips**

1. **GPU Acceleration**:
   - T4 GPU: Basic acceleration
   - V100 GPU: Optimal for medium datasets
   - A100 GPU: Optimal for large datasets

2. **Memory Optimization**:
   ```python
   # Batch size adjustment
   batch_size = 32 if torch.cuda.is_available() else 8
   
   # Memory clear
   torch.cuda.empty_cache() if torch.cuda.is_available() else None
   ```

3. **Data Size Adjustment**:
   - Testing: Under 1,000 vectors
   - Development: Under 10,000 vectors
   - Production: 100,000+ vectors

---

### 📚 Support Resources

#### 🔗 **External Links**

1. **Project Documentation**:
   - [Colab Troubleshooting Guide](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/guides/COLAB_TROUBLESHOOTING_GUIDE.md)
   - [Environment Setup Guide](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/guides/ENVIRONMENT_SETUP_GUIDE.md)
   - [Architecture Evolution Roadmap](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/ARCHITECTURE_EVOLUTION_ROADMAP.md)

2. **Technical Support**:
   - [GitHub Issues](https://github.com/miyauchikazuyoshi/InsightSpike-AI/issues)
   - [Discussions](https://github.com/miyauchikazuyoshi/InsightSpike-AI/discussions)

#### 📈 **Debug Information Collection**

**Information to include when reporting errors**:
```python
# System information
import sys, torch
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

# Error logs
import traceback
try:
    # Code that caused the error
    pass
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
```

#### 💪 **Community Performance**

**Result Sharing**:
- Share success stories and improvement suggestions on GitHub Discussions
- Contribute performance benchmark results to the community

**Feedback**:
- Report setup times and success rates
- Suggest new error patterns and solutions

---

### 🏁 **Success Checklist**

✅ GPU runtime enabled  
✅ All setup validation items successful  
✅ FAISS GPU test shows acceleration  
✅ Demo execution returns normal responses  
✅ Interactive test shows quality score > 0.5  
✅ All cells execute without errors  

**✨ When all are successful, InsightSpike-AI is working properly! ✨**